# 📊 Visualización de Grafos LangGraph

Este notebook te permite **visualizar la estructura** de cualquier grafo compilado.

## 🎯 Objetivo

- Ver la estructura de los grafos de codificación
- Comparar diferentes versiones
- Exportar diagramas para documentación

---


In [1]:
# ========================================
# 📦 SETUP INICIAL
# ========================================

import os
from pathlib import Path
from IPython.display import Image, display

# Configurar paths
project_root = Path.cwd().parent
os.chdir(project_root)

# Cargar variables de entorno
from dotenv import load_dotenv
load_dotenv()

print("✅ Setup completado")
print(f"📁 Directorio: {project_root}")


✅ Setup completado
📁 Directorio: c:\Users\ivan\Documents\cod-script


In [2]:
# ========================================
# 🔧 FUNCIÓN DE VISUALIZACIÓN
# ========================================

def visualizar_grafo(app, nombre="grafo"):
    """
    Visualiza un grafo LangGraph compilado.
    
    Args:
        app: Grafo compilado (resultado de workflow.compile())
        nombre: Nombre del grafo para identificarlo
    """
    print("="*60)
    print(f"📊 VISUALIZACIÓN: {nombre.upper()}")
    print("="*60)
    
    try:
        graph = app.get_graph()
        
        # Información básica
        print(f"\n📋 Información:")
        print(f"   Nodos: {len(graph.nodes)}")
        
        # Lista de nodos
        print(f"\n🔹 Nodos del grafo:")
        for i, node_id in enumerate(graph.nodes.keys(), 1):
            print(f"   {i}. {node_id}")
        
        # Vista ASCII
        print(f"\n{'='*60}")
        print("📊 DIAGRAMA ASCII")
        print("="*60)
        print(graph.draw_ascii())
        
        # Intentar generar PNG
        print(f"\n{'='*60}")
        print("🖼️  DIAGRAMA GRÁFICO")
        print("="*60)
        
        try:
            img_data = graph.draw_mermaid_png()
            display(Image(img_data))
            print("✅ Imagen generada correctamente")
        except Exception as e:
            print("⚠️  No se pudo generar imagen PNG")
            print(f"   Razón: {str(e)[:80]}...")
            print("\n📋 Código Mermaid (pega en https://mermaid.live):")
            print("="*60)
            mermaid_code = graph.draw_mermaid()
            print(mermaid_code)
            print("="*60)
            
            # Guardar en archivo
            output_dir = project_root / "notebooks" / "diagramas"
            output_dir.mkdir(exist_ok=True)
            output_file = output_dir / f"{nombre}.mmd"
            output_file.write_text(mermaid_code, encoding="utf-8")
            print(f"\n💾 Código guardado en: {output_file.relative_to(project_root)}")
            print("💡 Visita https://mermaid.live y pega el contenido del archivo")
        
        return graph
        
    except Exception as e:
        print(f"❌ Error: {e}")
        return None

print("✅ Función de visualización definida")


✅ Función de visualización definida


In [3]:
# ========================================
# 💾 EXPORTAR Y COMPARAR GRAFOS
# ========================================

def exportar_grafos(grafos_dict: dict):
    """
    Exporta múltiples grafos a archivos Mermaid y PNG.
    
    Args:
        grafos_dict: Diccionario {nombre: app_compilado}
    """
    output_dir = project_root / "notebooks" / "diagramas"
    output_dir.mkdir(exist_ok=True)
    
    print("="*60)
    print("💾 EXPORTANDO GRAFOS")
    print("="*60)
    
    for nombre, app in grafos_dict.items():
        print(f"\n🔹 {nombre}:")
        
        try:
            graph = app.get_graph()
            
            # Guardar Mermaid
            mermaid_code = graph.draw_mermaid()
            mermaid_file = output_dir / f"{nombre}.mmd"
            mermaid_file.write_text(mermaid_code, encoding="utf-8")
            print(f"   ✅ Mermaid: {mermaid_file.name}")
            
            # Intentar guardar PNG
            try:
                png_data = graph.draw_mermaid_png()
                png_file = output_dir / f"{nombre}.png"
                png_file.write_bytes(png_data)
                print(f"   ✅ PNG: {png_file.name}")
            except Exception as e:
                print(f"   ⚠️  PNG no disponible (instala mermaid-cli)")
                
        except Exception as e:
            print(f"   ❌ Error: {e}")
    
    print(f"\n{'='*60}")
    print(f"📁 Archivos guardados en: {output_dir.relative_to(project_root)}")
    print("="*60)


def comparar_grafos(grafos_dict: dict):
    """
    Compara métricas básicas de múltiples grafos.
    
    Args:
        grafos_dict: Diccionario {nombre: app_compilado}
    """
    print("="*60)
    print("📊 COMPARACIÓN DE GRAFOS")
    print("="*60)
    
    resultados = []
    
    for nombre, app in grafos_dict.items():
        try:
            graph = app.get_graph()
            num_nodos = len(graph.nodes)
            
            # Contar tipos de nodos
            nodos_normales = sum(1 for n in graph.nodes.keys() if not n.startswith("__"))
            
            resultados.append({
                "nombre": nombre,
                "total_nodos": num_nodos,
                "nodos_logicos": nodos_normales
            })
            
        except Exception as e:
            print(f"⚠️  Error procesando {nombre}: {e}")
    
    # Mostrar tabla
    if resultados:
        print(f"\n{'Grafo':<25} {'Total Nodos':<15} {'Nodos Lógicos':<15}")
        print("-" * 60)
        for r in resultados:
            print(f"{r['nombre']:<25} {r['total_nodos']:<15} {r['nodos_logicos']:<15}")
        
        # Análisis
        print(f"\n{'='*60}")
        print("📈 ANÁLISIS:")
        print("="*60)
        
        mas_simple = min(resultados, key=lambda x: x['nodos_logicos'])
        mas_complejo = max(resultados, key=lambda x: x['nodos_logicos'])
        
        print(f"\n✅ Más simple: {mas_simple['nombre']} ({mas_simple['nodos_logicos']} nodos)")
        print(f"⚙️  Más complejo: {mas_complejo['nombre']} ({mas_complejo['nodos_logicos']} nodos)")
        
        if mas_simple != mas_complejo:
            diferencia = mas_complejo['nodos_logicos'] - mas_simple['nodos_logicos']
            porcentaje = (diferencia / mas_simple['nodos_logicos']) * 100
            print(f"\n📊 Diferencia: +{diferencia} nodos (+{porcentaje:.1f}%)")

print("✅ Funciones de exportación y comparación definidas")


✅ Funciones de exportación y comparación definidas


---

## 💡 Cómo Usar Este Notebook

### **Opción 1: Importar Grafo desde Otro Notebook**

Ejecuta primero el notebook 03 o 05, luego:

```python
# Desde el notebook 03 (después de ejecutarlo)
%run 03_experimentacion_real.ipynb

# Visualizar
visualizar_grafo(app, "grafo_v1_real")
```

---

### **Opción 2: Copiar Variable de Grafo**

Si ya ejecutaste otro notebook en esta sesión:

```python
# Asume que 'app' ya existe de otro notebook ejecutado
visualizar_grafo(app, "mi_grafo")
```

---

### **Opción 3: Importar desde Producción** (cuando esté migrado)

```python
from backend.src.cod_backend.core.grafo import app_produccion
visualizar_grafo(app_produccion, "produccion_v1")
```

---

### **Ejemplos de Uso Completo:**

#### **Visualizar un solo grafo:**
```python
visualizar_grafo(app, "nombre_descriptivo")
```

#### **Exportar múltiples grafos:**
```python
exportar_grafos({
    "grafo_v1": app,
    "grafo_v2": app_v2,
    "grafo_optimizado": app_opt
})
```

#### **Comparar grafos:**
```python
comparar_grafos({
    "V1 Simple": app,
    "V2 Dividido": app_v2
})
```

---
